# Bond Option Pricing using Various Models

Value an option on a coupon paying bond using the Hull-White, Black-Derman-Toy and Black-Karasinski model

In [72]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [73]:
from financepy.finutils import *
from financepy.market.curves import *
from financepy.models import FinModelRatesBK
from financepy.models import FinModelRatesBDT
from financepy.models import FinModelRatesHW
from financepy.products.bonds import *

## Set up the Discount Curve

In [74]:
settlementDate = FinDate(1, 12, 2019)

Set up discount curve

In [75]:
rate = 0.10
dcType = FinDayCountTypes.THIRTY_360_BOND
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
discountCurve = FinDiscountCurveFlat(settlementDate, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [76]:
issueDate = FinDate(1,12,2018)
maturityDate = issueDate.addTenor("3Y")
coupon = 0.10
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualType = FinDayCountTypes.THIRTY_360_BOND
bond = FinBond(issueDate, maturityDate, coupon, frequencyType, accrualType)

Let's first price the bond on the libor curve

In [77]:
cp = bond.cleanPriceFromDiscountCurve(settlementDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(settlementDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   105.000


In [78]:
settlementDate

SUN 01 DEC 2019

In [79]:
bond.printFlows(settlementDate)

SUN 01 DEC 2019          5.00 
MON 01 JUN 2020          5.00 
TUE 01 DEC 2020          5.00 
TUE 01 JUN 2021          5.00 
WED 01 DEC 2021        105.00 


# Create the Bond Options

Then define and create the option

In [80]:
expiryDate = settlementDate.addTenor("18m")
strikePrice = 95.0
faceAmount = 100.0

In [81]:
europeanCallBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.EUROPEAN_CALL)
americanCallBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.AMERICAN_CALL)

In [82]:
europeanPutBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.EUROPEAN_PUT)
americanPutBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [83]:
cp = bond.cleanPriceFromDiscountCurve(expiryDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(expiryDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   105.000


In [84]:
df = discountCurve.df(expiryDate)
callIntrinsic = max(cp - strikePrice, 0) * df
putIntrinsic = max(strikePrice - cp, 0) * df

In [85]:
callIntrinsic

4.319187992657368

In [86]:
putIntrinsic

0.0

## Tree Models

Set the model parameters, start with zero vol.

In [87]:
numSteps = 200

In [88]:
sigma = 0.00001
a = 0.1
modelHW = FinModelRatesHW(sigma*rate, a, numSteps)
modelBK = FinModelRatesBK(sigma, a, numSteps)
modelBDT = FinModelRatesBDT(sigma, numSteps)

Check some limits - the call option should be worth the intrinsic.

In [89]:
ec = europeanCallBondOption.value(settlementDate, discountCurve, modelHW)
ac = americanCallBondOption.value(settlementDate, discountCurve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.31862
American Call Value:   4.99399


The put option too.

In [90]:
ep = europeanPutBondOption.value(settlementDate, discountCurve, modelHW)
ap = americanPutBondOption.value(settlementDate, discountCurve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00000


Now we turn on the volatility.

In [91]:
sigma = 0.20
a = 0.1
modelHW = FinModelRatesHW(sigma*rate, a, numSteps)
modelBK = FinModelRatesBK(sigma, a, numSteps)
modelBDT = FinModelRatesBDT(sigma, numSteps)

In [92]:
ec = europeanCallBondOption.value(settlementDate, discountCurve, modelHW)
ac = americanCallBondOption.value(settlementDate, discountCurve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.31863
American Call Value:   5.28080


In [93]:
ep = europeanPutBondOption.value(settlementDate, discountCurve, modelHW)
ap = americanPutBondOption.value(settlementDate, discountCurve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00457


Copyright (c) Dominic O'Kane 2020